In [ ]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook
from tweepy import OAuthHandler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/Manu Master/twitterapi.txt') as f:
  lines = f.read().splitlines()
  consumer_key = lines[0]
  consumer_secret = lines[1]
  access_key = lines[3]
  access_secret = lines[4]
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
#api = tweepy.API(auth)
api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
search_words = "#covidvaccination OR #AstraZeneca OR #curevac OR #Moderna OR #Pfizer OR #BioNTech OR #covishield OR #covaxin OR #Sinopharm OR #SputnikV -filter:retweets"

#search_words = "#covid19 -filter:retweets"
date_since = "2021-04-01"
date_till = "2021-05-11"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since
              #until=date_till
              ).items(900)

In [ ]:
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

900it [00:36, 24.69it/s]


In [ ]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 900


In [ ]:
from tqdm import tqdm, notebook
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 900/900 [04:33<00:00,  3.29it/s]


In [ ]:
#@title Standardtext für Titel
import pandas as pd
#f = '/content/drive/My Drive/Manu Master/twitter_data_analysis.csv'
#tweets_df.to_csv(f)
tweets_old_df = pd.read_csv("/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv")
print(f"past tweets: {tweets_old_df.shape}")
fileold = '/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'
if(os.path.exists(fileold) and os.path.isfile(fileold)):
  os.remove(fileold)
  print("file deleted")
else:
  print("file not found")

past tweets: (209345, 9)
file deleted


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:

tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 900 past tweets: 209345 all tweets: 210245


In [ ]:
tweets_all_df.drop_duplicates(subset = ["user_name", "text"], keep= 'first', inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (210239, 9)


In [ ]:
import os
# first check whether file exists or not
fl = '/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv'
tweets_all_df.to_csv(fl,index=False) 
# first check whether file exists or not
# calling remove method to delete the csv file
# in remove method you need to pass file name and type
file = '/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
  print("file deleted")
else:
  print("file not found")

file not found


In [ ]:
import shutil
original = r'/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv'
target = r'/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'

shutil.copyfile(original, target)


'/content/drive/My Drive/Manu Master/twitter_data_analysisII.csv'

In [ ]:
tweets_saved_df = pd.read_csv("/content/drive/My Drive/Manu Master/twitter_data_analysisI.csv")
print(f"Last saved tweets: {tweets_saved_df.shape}")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Last saved tweets: (210239, 9)


**cleaning the row data from twitter: (i) select only those tweets with userlocation, usercreated and tweet text to avoid ambigious data; (ii) remove non ascii characters from twitter text (iii) remove duplicates data in username, user_created or text and reset index**

In [ ]:
import numpy as np
df_new=tweets_saved_df[tweets_saved_df["user_location"].notnull()] 
df_ultranew=df_new[df_new['user_location'].map(lambda x: x.isascii())]

df_new=df_ultranew[df_ultranew["text"].notnull()] 
#df_ultranew=df_new[df_new['text'].map(lambda x: x.isascii())]
modified_df = df_ultranew.dropna(subset=['user_location'])
modified_df['user_created'].replace('', np.nan, inplace=True)
modified_df = df_ultranew.dropna(subset=['user_created'])
modified_df = modified_df.drop_duplicates(subset=['user_name','user_created','text'])
modified_df= modified_df.reset_index(drop=True)

In [ ]:
import os
fl = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetsI.csv'
modified_df.to_csv(fl,index=False) 
file = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetsbackupI.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
  print("file deleted")
else:
  print("file not found")

file deleted


In [ ]:
import shutil
original = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetsI.csv'
target = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetsbackupI.csv'
shutil.copyfile(original, target)
print("file copied and backed up")

file copied and backed up


In [ ]:
modified_df[-50:]

,user_name,user_location,user_description,user_created,user_verified,date,text,hashtags,is_retweet
117469,NSM Today,"East Orlando, FL",NSM Today's collective of student journalists produce accountable and committed stories serving the @UCF community. #ForUCFByUCF,2015-12-11 16:19:29,False,2021-08-29 02:02:00,"As the #FDA grants full approval of the Pfizer-BioNTech #COVID19 vaccine, @UCF is trying to encourage non-vaccinated students to get the jab. \n\n#CovidVaccine #Pfizer #UCF #news \n\nhttps://t.co/...","['FDA', 'COVID19']",False
117470,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,False,2021-08-29 02:00:24,"45+ #BBMP #Bengaluru #CovidVaccine Availability for 29/08 at 07:30AM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose1:607, Dose2:671\n#COVAXIN - Dose1:267, Dose2:304","['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",False
117471,J_N@sh,Australia,"Countering alternative realities, Murdoch/Newscorp lies, Morrison marketing & spin. Campaigning for climate action now & climate justice.",2020-03-04 02:11:26,False,2021-08-29 01:59:10,@jenfromthelake @ProfessorChook @ScottMorrisonMP Why would &gt;60s want a different vaccine?! \n\n#AstraZeneca or death\n\nIs exactly the same as\n\n#Pfizer/AZ or death\n\nPfizer is needed 2 SAFEL...,['AstraZeneca'],False
117472,Nitendra Singh नितेन्द्र सिंह,New Delhi,"Works at @DDNewsLive. Covers #Health, #Publicpolicy, #Socialissues, #Travel, #Foodie @IIMC alum\n RT is not endorsement. Tweets personal.",2012-02-13 05:54:17,True,2021-08-29 01:56:40,"#COVID19India \n\nUnion Health Minister @mansukhmandviya to release first batch of #COVAXIN made in @BharatBiotech's plant at Ankleshwar, #Gujarat, today at 10:50 am \n\n@MoHFW_INDIA \n#IndiaFight...","['COVID19India', 'COVAXIN']",False
117473,Wes,"Alabama, USA",Host of the Fantasy Impact Today podcast @FIToday_ //Truth Seeker est. 2018 // NBA Live 99 Champion,2008-06-11 02:51:25,False,2021-08-29 01:50:56,Does anyone know what is going on with the neck of the #Pfizer CEO?\n\nI've never seen a neck do that before. https://t.co/F58PhdSQKq,['Pfizer'],False
117474,Leah Galvin ✳✳🍎🍒🥑🥦🥕🌽🌰🍓,"Tasmania, Australia",Preoccupied with #foodsystems #foodsecurity #climatechange #Tasmanian Churchill Fellowship Recipient 2019,2011-08-19 05:09:17,False,2021-08-29 01:45:34,There is a pop up walk in #covidvax clinic at Maq2 today Sunday. #Pfizer Everyone 12+ can get vaccinated. Let's do it! #hobart #covid19tas https://t.co/KLjSrXuTPB,"['covidvax', 'Pfizer']",False
117475,Diana DJ,Gadigal Land,A frustrated idealist// #ranter #family #travel #food #movies #TV #sport #Portugal🇵🇹 #Australia🇭🇲 #MollyTheCocker @bcurrall80 //Views are my own,2011-10-08 05:53:03,False,2021-08-29 01:40:45,There are #Pfizer appointments available TODAY! https://t.co/E2Ji9btPEp,['Pfizer'],False
117476,yeetedracer // VIVA RONALDO 🔴,nope,ManUTD / UtahJazz / SF49ers / SFGiants (84-45) / RedBullRacing / Sidemen // and tweets random stuff...,2020-05-29 11:46:45,False,2021-08-29 01:39:40,"Someone is spreading false information that the #pfizer #vaccines contain ingredients of HIV. That man named ""Doc Ron"" needs to go to jail.\n\nAlso, it's in the Philippines.\n\nNow my dad is belie...","['pfizer', 'vaccines']",False
117477,Women for Australia (W4A),Brisbane/Turrbal country,We are the 51 percent.,2021-03-04 00:03:27,False,2021-08-29 01:39:29,"""I trust my immune system""\n\nDo you, Sapphire? Good for you. Trust it in silence and stop being an ableist bishhhh.\n\n#GetVaccinated #pfizer #AstraZeneca #TrustScience\n #COVID19 #",NaN,False
117478,JGalicot1,Tikkun Olam,CoFounder-TijuanaInnovadora-The efforts of the citizens of BC have rendered results;TODAY WE LIVE @ PEACE.LET'S KEEP GOING.#FilmTank #TikunOlam #WeizmannScience,2018-04-23 01:20:21,False,2021-08-29 01:38:37,Previous Covid Prevents #Delta Infection Better Than #Pfizer Shot - ⁦@business⁩ #Covid https://t.co/tTeZigP5Rq,"['Delta', 'Pfize

***combining internet data***


In [ ]:
tweets_frominternet_df = pd.read_csv("/content/drive/My Drive/Manu Master/dataset/vaccination_all_tweets.csv")
#tweets_frominternet_df = pd.read_csv("D://vaccination_all_tweets.csv")
tweets_frominternet_df = tweets_frominternet_df.drop(['source','retweets','favorites'], axis = 1)
print(f"Tweets from Internet: {tweets_frominternet_df.shape}")
tweets_frominternet_df[-5:]

Tweets from Internet: (177016, 13)


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,is_retweet
177011,1428288192210374662,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,24,0,0,False,2021-08-19 09:30:15,18-44 #URBAN #Bengaluru #CovidVaccine Availability for 19/08 at 03:00PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose… https://t.co/C9116FQSvC,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",False
177012,1428280699883200512,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,24,0,0,False,2021-08-19 09:00:29,"45+ #BBMP #Bengaluru #CovidVaccine Availability for 19/08 at 02:30PM\nFREE Slots \n#COVISHIELD - Dose1:9, Dose2:2… https://t.co/plTlxFM6Bp","['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",False
177013,1428280655767502850,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,24,0,0,False,2021-08-19 09:00:19,45+ #URBAN #Bengaluru #CovidVaccine Availability for 19/08 at 02:30PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose1:… https://t.co/PnrO67Ts47,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",False
177014,1428273162270035969,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,24,0,0,False,2021-08-19 08:30:32,"18-44 #BBMP #Bengaluru #CovidVaccine Availability for 19/08 at 02:00PM\nFREE Slots \n#COVISHIELD - Dose1:1, Dose2:0\nP… https://t.co/XVFLUSUypq","['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",False
177015,1428273118229925889,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,24,0,0,False,2021-08-19 08:30:22,18-44 #URBAN #Bengaluru #CovidVaccine Availability for 19/08 at 02:00PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose… https://t.co/CaY3SvBesk,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",False


In [ ]:
import numpy as np
df_newint=tweets_frominternet_df[tweets_frominternet_df["user_location"].notnull()] 
df_ultranewint=df_newint[df_newint['user_location'].map(lambda x: x.isascii())]

df_newint=df_ultranewint[df_ultranewint["text"].notnull()] 
#df_ultranew=df_new[df_new['text'].map(lambda x: x.isascii())]
modified_dfint = df_ultranewint.dropna(subset=['user_location'])
modified_dfint['user_created'].replace('', np.nan, inplace=True)
modified_dfint = df_ultranewint.dropna(subset=['user_created'])
modified_dfint = modified_dfint.drop_duplicates(subset=['user_name','user_created','text'])
modified_dfint= modified_dfint.reset_index(drop=True)
modified_dfint.drop(['id', 'user_followers','user_friends','user_favourites'], axis = 1,inplace=True)

In [ ]:
modified_dfint[:5]

,user_name,user_location,user_description,user_created,user_verified,date,text,hashtags,is_retweet
0,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],False
1,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",NaN,False
2,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",False
3,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radio Network. Weeknights 7 Pacific-10 Eastern - Email comments/ideas to charles@charlesadlertonight.ca",2008-09-10 11:28:53,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4",NaN,False
4,Dee,"Birmingham, England","Gastroenterology trainee, Clinical Research Fellow in IBD, mother to human and fur baby, Canadian in Britain",2020-01-26 21:43:12,False,2020-12-12 20:11:42,Does anyone have any useful advice/guidance for whether the COVID vaccine is safe whilst breastfeeding?… https://t.co/EifsyQoeKN,NaN,False


saving data for later use with backup

In [ ]:
import os
fl = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetsprada.csv'
modified_dfint.to_csv(fl,index=False) 
file = '/content/drive/My Drive/Manu Master/dataset/cleandatatweetspradabackup.csv'
if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)
  print("file saved/deleted and backedup")
else:
  print("file not found")
original = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetsprada.csv'
target = r'/content/drive/My Drive/Manu Master/dataset/cleandatatweetspradabackup.csv'
shutil.copyfile(original, target)

file saved/deleted and backedup


'/content/drive/My Drive/Manu Master/dataset/cleandatatweetspradabackup.csv'



---

**End of this part... go to geolocation notebook**